In [1]:
import time
notebookstart= time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold

# Tf-Idf
#import nltk
#nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 

# LDA
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

# Viz
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string

NFOLDS = 5
SEED = 1234
VALID = True

/home/stanaya/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
## scikit-learn classifierのラッパークラス
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None, seed_bool = True):
        if(seed_bool == True):
            params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

In [3]:
# OOF : Out Of Fold
def get_oof(clf, x_train, y, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        print('\nFold {}'.format(i))
        x_tr = x_train[train_index]
        y_tr = y[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [4]:
def cleanName(text):
    try:
        textProc = text.lower()
        textProc = re.sub('[!@#$_“”¨«»®´·º½¾¿¡§£₤‘’]', '', textProc)
        textProc = " ".join(textProc.split())
        return textProc
    except: 
        return "name error"
    
    
def rmse(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power((y - y0), 2)))

In [5]:
print("\nData Load Stage")
training = pd.read_csv('/home/stanaya/.kaggle/competitions/avito-demand-prediction/train.csv', index_col = "item_id", parse_dates = ["activation_date"])
traindex = training.index
testing = pd.read_csv('/home/stanaya/.kaggle/competitions/avito-demand-prediction/test.csv', index_col = "item_id", parse_dates = ["activation_date"])
testdex = testing.index

ntrain = training.shape[0]
ntest = testing.shape[0]

kf = KFold(ntrain, n_folds=NFOLDS, shuffle=True, random_state=SEED)

y = training.deal_probability.copy()
training.drop("deal_probability",axis=1, inplace=True)
print('Train shape: {} Rows, {} Columns'.format(*training.shape))
print('Test shape: {} Rows, {} Columns'.format(*testing.shape))


Data Load Stage
Train shape: 1503424 Rows, 16 Columns
Test shape: 508438 Rows, 16 Columns


In [6]:
# 学習データとテストデータを統合
print("Combine Train and Test")
df = pd.concat([training,testing],axis=0)
del training, testing
gc.collect()
print('\nAll Data shape: {} Rows, {} Columns'.format(*df.shape))

Combine Train and Test

All Data shape: 2011862 Rows, 16 Columns


In [7]:
print("Feature Engineering")
df["price"] = np.log(df["price"]+0.001)
df["price"].fillna(df.price.mean(),inplace=True)
df["image_top_1"].fillna(-999,inplace=True)

Feature Engineering


In [8]:
df.head()

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
item_id,,,,,,,,,,,,,,,,
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",5.991467,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0
2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",8.006368,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0
ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",8.294050,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0
02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,7.696213,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0
7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,10.596635,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0


In [9]:
# 曜日、週の情報を追加
print("\nCreate Time Variables")
df["Weekday"] = df['activation_date'].dt.weekday
df["Weekd of Year"] = df['activation_date'].dt.week
df["Day of Month"] = df['activation_date'].dt.day


Create Time Variables


In [10]:
# Create Validation Index and Remove Dead Variables
# training と validation のindexを抽出(item_id)
training_index = df.loc[df.activation_date<=pd.to_datetime('2017-04-07')].index
validation_index = df.loc[df.activation_date>=pd.to_datetime('2017-04-08')].index
df.drop(["activation_date","image"],axis=1,inplace=True) # imageの情報は使わない

# categoricalな特徴量を抽出する
print("\nEncode Variables")
categorical = ["user_id","region","city","parent_category_name","category_name","user_type","image_top_1","param_1","param_2","param_3"]
print("Encoding :",categorical)

# Encoder:
# 文字データを離散数値に変換
lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col].fillna('Unknown')
    df[col] = lbl.fit_transform(df[col].astype(str))


Encode Variables
Encoding : ['user_id', 'region', 'city', 'parent_category_name', 'category_name', 'user_type', 'image_top_1', 'param_1', 'param_2', 'param_3']


In [11]:
#df.head()

In [12]:
print("\nText Features")

# Feature Engineering 

# Meta Text Features
textfeats = ["description", "title"]
# 句読点、括弧の数を特徴量にする
df['desc_punc'] = df['description'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# タイトルと説明文を正規化する
df['title'] = df['title'].apply(lambda x: cleanName(x))
df["description"]   = df["description"].apply(lambda x: cleanName(x))

# タイトルと説明文の全単語数とユニークな単語種類、さらにそれらの比を撮ったものを加える
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('missing') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words


Text Features


In [13]:
## 
df_text = df[["title", "description"]]

In [14]:
dic_replace_title = {',': ' ', '(':' ', ')': ' ', '.':' '}
dic_replace_desc = {',': ' ', '(':' ', ')': ' ', '.':' ', '\n': '', '\r': '', '/': ''}

def replaceTitleSomeCharSplit(s):
    return list(filter(None, s.translate(str.maketrans(dic_replace_title)).split(" ")))

def replaceDescSomeCharSplit(s):
    return list(filter(None, s.translate(str.maketrans(dic_replace_desc)).split(" ")))


def replaceTitleSomeChar(s):
    return s.translate(str.maketrans(dic_replace_title))

def replaceDescSomeChar(s):
    return s.translate(str.maketrans(dic_replace_desc))

In [15]:
df_text["title_split"] = df_text["title"].map(replaceTitleSomeChar)
df_text["desc_split"] = df_text["description"].map(replaceDescSomeChar)

/home/stanaya/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/stanaya/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
df_text_split = df_text[['title_split', 'desc_split']]

In [17]:
corpusTitle = df_text_split['title_split'].as_matrix()

In [40]:
len(corpusTitle)

2011862

In [18]:
##corpusTitle

gc.collect()

0

In [72]:
## Topic Model
print("\nTopic Modeling!")
NUM_TOPICS = 300
russian_stop = set(stopwords.words('russian'))
def get_col(col_name): return lambda x: x[col_name]

 
#vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
#                            stop_words='russian', lowercase=True)
vectorizer = CountVectorizer(
            stop_words = russian_stop,
            max_df=10,
            min_df=10)
data_title_vectorized = vectorizer.fit_transform(corpusTitle)


Topic Modeling!


In [73]:
features = vectorizer.get_feature_names()
print(len(features))
print(features)

1961
['033', '034', '045', '046', '051', '060', '065', '0a', '0gb', '0мес', '1003', '1005pxd', '100a', '1025c', '105l', '10600u', '10вт', '1122', '1132', '1142', '1170', '11кг', '1205', '1260', '1300w', '1314', '13mp', '1457', '145см', '1460', '14т', '1502', '150к', '1545', '1550', '15g', '15m', '15лет', '15мм', '15шт', '160x200', '1615', '1680', '1701', '1744', '1804', '180mm', '1811', '1843', '1866', '1871', '1925г', '1937г', '1964г', '1969г', '1984г', '1v', '1м3', '1п', '1х1', '1х2', '2000gb', '200d', '200м', '22дюйма', '2346', '24часа', '259', '2601', '26м', '2710', '2760', '279', '2835', '284', '286', '29см', '2usb', '2симки', '300р', '305х76', '3121', '320hdd', '32гига', '33021', '330232', '33ghz', '346', '3525', '35мм', '36v', '371', '376', '377', '3770k', '37рр', '397', '3e', '3w', '3в1и2в1разные', '3гига', '3р', '3с', '4000mah', '4010u', '4040', '41размер', '44рр', '45л', '464', '4690', '4х1', '500руб', '5045', '50г', '50руб', '5150', '520m', '53366', '5440а5', '5502', '552g',

In [ ]:
## Topic Model
#print("\nTopic Modeling!")
#NUM_TOPICS = 10
#russian_stop = set(stopwords.words('russian'))
#def get_col(col_name): return lambda x: x[col_name]

 
#vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
#                            stop_words='russian', lowercase=True)
#vectorizer = CountVectorizer(
 #           ngram_range=(1, 2),
#            stop_words = russian_stop,
            #max_features=7000,
  #          preprocessor=get_col('title'))
#data_vectorized = vectorizer.fit_transform(df.to_dict('records'))

In [74]:
# Build a Latent Dirichlet Allocation Model
#lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
#lda_Z = lda_model.fit_transform(data_title_vectorized)
#print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
#nmf_model = NMF(n_components=NUM_TOPICS)
#nmf_Z = nmf_model.fit_transform(data_title_vectorized)
#print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_title_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
#print(lda_Z[0])
#print(nmf_Z[0])
print(lsi_Z[0])

(2011862, 300)
[-2.32043088e-15  1.61380479e-15 -2.36053952e-15 -1.29007543e-15
  4.59260823e-16 -1.11670465e-15 -8.99014884e-16 -8.22566672e-16
 -3.80345118e-15 -3.46956031e-15  5.78204570e-16  6.44242775e-16
  2.47687138e-15  6.98549285e-15  2.50594332e-15  1.98742074e-15
  1.09357744e-14  1.34581280e-14  3.31709827e-15 -5.02353016e-15
  4.55203860e-15 -3.30912301e-15 -9.31913588e-15  1.49831369e-15
  3.56195818e-15 -3.12443290e-15 -3.62786488e-15 -6.61947279e-16
 -3.23621827e-15 -3.14804204e-15  2.33349375e-15 -8.90108753e-15
 -1.40243651e-15 -6.70515454e-15 -5.55242160e-15  9.05965416e-15
  5.47826632e-15  1.77617727e-15  2.54829654e-15 -4.49752277e-16
  9.91732543e-16  1.69310978e-15  6.93563030e-15 -5.89290608e-15
  1.42183647e-15  3.73642984e-15  4.02194099e-15 -1.16243939e-14
 -2.66644775e-15 -8.31784318e-16 -3.98250374e-15  4.56005597e-15
 -2.82956840e-15 -4.97750963e-15 -3.20028014e-15  7.62264634e-16
 -2.22257430e-15 -6.33889038e-16 -2.18254593e-15  5.38012264e-17
  1.608536

In [78]:
print(sum(lsi_Z[100]))

-3.353109463128523e-14


In [ ]:
print("\n[TF-IDF] Term Frequency Inverse Document Frequency Stage")
# ロシア語のストップワード（自然言語処理する際に一般的で役に立たない等の理由で処理対象外とする単語）の除去
russian_stop = set(stopwords.words('russian'))

tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}

In [ ]:
def get_col(col_name): return lambda x: x[col_name]
##I added to the max_features of the description. It did not change my score much but it may be worth investigating
vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=17000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('title',CountVectorizer(
            ngram_range=(1, 2),
            stop_words = russian_stop,
            #max_features=7000,
            preprocessor=get_col('title')))
    ])

In [ ]:
vectorizer 

In [ ]:
start_vect=time.time()

#Fit my vectorizer on the entire dataset instead of the training rows
#Score improved by .0001
vectorizer.fit(df.to_dict('records'))

ready_df = vectorizer.transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

In [ ]:
# Drop Text Cols
textfeats = ["description", "title"]
df.drop(textfeats, axis=1,inplace=True)

In [ ]:
# Combine Dense Features with Sparse Text Bag of Words Features
X = df.loc[traindex,:].values
testing = df.loc[testdex,:].values
tfvocab = df.columns.tolist()
for shape in [X,testing]:
    print("{} Rows and {} Cols".format(*shape.shape))
del df
gc.collect();

In [ ]:
print("\nModeling Stage")
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=1234)

gc.collect();

In [ ]:
print("Light Gradient Boosting Regressor")
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    # 'max_depth': 15,
    'num_leaves': 300,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.85,
    # 'bagging_freq': 5,
    'learning_rate': 0.019,
    'verbose': 0
}  

In [ ]:
if VALID == True:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.10, random_state=1234)
        
    # LGBM Dataset Formatting 
    lgtrain = lgb.Dataset(X_train, y_train,
                    feature_name=tfvocab,
                    categorical_feature = categorical)
    lgvalid = lgb.Dataset(X_valid, y_valid,
                    feature_name=tfvocab,
                    categorical_feature = categorical)
    del X, X_train; gc.collect()
    
    # Go Go Go
    lgb_clf = lgb.train(
        lgbm_params,
        lgtrain,
        num_boost_round=n_rounds,
        valid_sets=[lgtrain, lgvalid],
        valid_names=['train','valid'],
        early_stopping_rounds=50,
        verbose_eval=100
    )
    print("Model Evaluation Stage")
    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_valid, lgb_clf.predict(X_valid))))
    del X_valid ; gc.collect()

else:
    # LGBM Dataset Formatting 
    lgtrain = lgb.Dataset(X, y,
                    feature_name=tfvocab,
                    categorical_feature = categorical)
    del X; gc.collect()
    # Go Go Go
    lgb_clf = lgb.train(
        lgbm_params,
        lgtrain,
        num_boost_round=910,
        verbose_eval=100
    )

In [ ]:
print("Model Evaluation Stage")
lgpred = lgb_clf.predict(testing) 

In [ ]:
#Mixing lightgbm with ridge. I haven't really tested if this improves the score or not
#blend = 0.95*lgpred + 0.05*ridge_oof_test[:,0]
lgsub = pd.DataFrame(lgpred,columns=["deal_probability"],index=testdex)
lgsub['deal_probability'].clip(0.0, 1.0, inplace=True) # Between 0 and 1
lgsub.to_csv("lgsub.csv",index=True,header=True)
#print("Model Runtime: %0.2f Minutes"%((time.time() - modelstart)/60))
print("Notebook Runtime: %0.2f Minutes"%((time.time() - notebookstart)/60))